In [1]:
import numpy as np

import defines
from classic_MF import MF
# from classic_bin_MF import BinaryMF
from data_handler import Data_Handler
from embedded_QRS import embedded_QRS_model1
from embedded_QRS_model2 import embedded_QRS_model2
from embedded_QRS_model3 import embedded_QRS_model3
from embedded_QRS_model4 import embedded_QRS_model4
from embedded_QRS_model5 import QRS_user_items
from QRS_user_group_global import QRS_user_group_global_items
from QRS_embedded_item import QRS_embedded_layer_item_layer
from embedded_QRS_MF import QMF
# from hist_removal_per_user import hist_removal_per_user
from QRS_hist_removal_per_user import QRS_hist_removal_per_user
from QRS_amp_amp_per_user import QRS_amplitude_amp
from random_RS import random_RS
from popularity_RS import popularity_RS
import visualiser
from embedded_QRS import randomize_init_params
from Q_data_prep_nn import improve_entanglemet
from entangelment_testing import entangling_tester
from itertools import combinations
from nbconvert import HTMLExporter
import codecs
import nbformat



def find_optimized_items_indexing():
    N_iters_per_set = 3
    possible_states = 2**defines._NUM_OF_QUBITS_TO_OPTIMIZE
    set_size = 2

    all_sets = list(combinations(list(range(possible_states)), set_size))
    #print(all_sets)
    print("total sets", len(all_sets))
    weigth_graph = np.zeros((possible_states, possible_states))

    for si, s in enumerate(all_sets):
        error = 0
        expected_probs = np.zeros(possible_states)
        expected_probs[list(s)] = 1/set_size
        for i in range(N_iters_per_set):
            ET = entangling_tester(expected_probs, 10, 1)
            ET.train()
            error += ET.test_entangling()
        error = error/N_iters_per_set # taking average
        print('set {ind}-{sett}: {err:.4f}'.format(ind=si, sett=s, err=error))
        weigth_graph[s[0]][s[1]] = error

    with open('./exported_data/'+str(defines._NUM_OF_QUBITS_TO_OPTIMIZE)+'_opt_score_graph.npy', 'wb') as f:
        np.save(f, weigth_graph)


# input: reco_hit_index arr - for every recommendation - where the LOO item was
# output: HR@K arr
def create_HRK(hit_arr):
    hits_ind = np.zeros(11)
    for hit in hit_arr:
        if hit <= 10:
            hits_ind[hit] += 1
    _hit_arr = np.cumsum(hits_ind)
    _hit_arr = _hit_arr/len(hit_arr)
    return _hit_arr



# input:  uninteracted movies array
#         LOO item
#         scores array - in corelative order to uninteracted movies
# output: the index of the LOO item in the scores array
def get_LOO_index_in_scores_array(uninteracted_movies, LOO_item, reco_scores, search_reco_item_within_all_items=0):
    if search_reco_item_within_all_items == 0:
        t = np.array(uninteracted_movies)
        desired_inter_index_pos = np.where(t == LOO_item)[0][0]
    else:
        desired_inter_index_pos = LOO_item
    hit_index = (np.where(reco_scores.argsort()[::-1][:len(reco_scores)]==desired_inter_index_pos)[0])[0]+1
    return hit_index


# input:    1. list of tuples
#               A. userID - who we removed the interaction from
#               B. moviedId - which was removed from the user
#           2. if num_of_uninter_per_user == 0 - than taking all uninteracted movies
# output:   1. list of triples:
#               A. user index (encoded)
#               B. the movie index which removed (encoded)
#               C. list of uninteracted movies - contains the removed interaction movie (encoded)
def create_recommendation_sets(LOO, dh, num_of_uninter_per_user = 0):
    recommendation_sets = []
    for user, movieId in LOO:
        # getting all uninteracted movies
        uninteracted_movies = dh.get_uninteracted_movieId_to_user(user)

        # sample X of the uninteracted movies
        if num_of_uninter_per_user != 0:
            uninteracted_movies = np.random.choice(uninteracted_movies, num_of_uninter_per_user, replace=False)
        uninteracted_movies = [dh.convert_movieId_to_movie_encode(x) for x in uninteracted_movies]

        # convert the LOO movieId to encoded movie and add it to list
        LOO_encoded_movie = dh.convert_movieId_to_movie_encode(movieId)
        uninteracted_movies.append(LOO_encoded_movie)
        uninteracted_movies = list(set(uninteracted_movies))

        user_encoded = dh.convert_userId_to_user_encode(user)
        recommendation_sets.append((user_encoded, LOO_encoded_movie, uninteracted_movies))
    print("done creating recommendation sets")
    recommendation_sets = sorted(recommendation_sets)
    return recommendation_sets



def TEST_MODEL(recommendation_sets, MODEL, search_reco_item_within_all_items=0):
    reco_hit_index = []
    total_noise = 0
    for user, removed_movie, uninter_movies in recommendation_sets:
        # get recommendations from the model - on uninter_movies list
        reco_scores = MODEL.get_recommendation(user, uninter_movies, removed_movie)
        # getting the index of the LOO item from the recommendations
        hit_index = get_LOO_index_in_scores_array(uninter_movies, removed_movie, reco_scores, search_reco_item_within_all_items)
        reco_hit_index.append(hit_index)

        if search_reco_item_within_all_items:
            total_noise += sum(reco_scores[uninter_movies])
        else:
            total_noise += sum(reco_scores)


    reco_hit_index = np.array(reco_hit_index)
    HRK = create_HRK(reco_hit_index)
    print("---Total noise:", total_noise,"---")
    return HRK


def export_data(name_in):
    name = str(name_in)
    out_recommendation_sets = np.empty(len(recommendation_sets), dtype=object)
    out_recommendation_sets[:] = recommendation_sets
    with open('./exported_data/'+name+'_recommendation_sets_good.npy', 'wb') as f:
        np.save(f, out_recommendation_sets)
    with open('./exported_data/'+name+'_R_df_as_numpy_good.npy', 'wb') as f:
        np.save(f, R_df_as_numpy)
#     with open('./exported_data/'+name+'_QRS_init_params.npy', 'wb') as f:
#         np.save(f, QRS_init_params)
    with open('./exported_data/'+name+'_user_embedded_vecs.npy', 'wb') as f:
        np.save(f, user_embedded_vecs)
    with open('./exported_data/'+name+'_item_embedded_vecs.npy', 'wb') as f:
        np.save(f, item_embedded_vecs)
    with open('./exported_data/'+name+'_HRK_MF.npy', 'wb') as f:
        np.save(f, HRK_MF_SDG)
    with open('./exported_data/'+name+'_HRK_RAND_RECO.npy', 'wb') as f:
        np.save(f, HRK_RAND_RECO)   
    with open('./exported_data/'+name+'_HRK_POP_RECO.npy', 'wb') as f:
        np.save(f, HRK_POP_RECO)   

        
def load_data(name_in):
    name = str(name_in)
    with open('./exported_data/'+name+'_recommendation_sets_good.npy', 'rb') as f:
        out_recommendation_sets = np.load(f, allow_pickle=True)
    with open('./exported_data/'+name+'_R_df_as_numpy_good.npy', 'rb') as f:
        R_df_as_numpy = np.load(f, allow_pickle=True)
#     with open('./exported_data/'+name+'_QRS_init_params.npy', 'rb') as f:
#         QRS_init_params = np.load(f, allow_pickle=True)
    with open('./exported_data/'+name+'_user_embedded_vecs.npy', 'rb') as f:
        user_embedded_vecs = np.load(f, allow_pickle=True)
    with open('./exported_data/'+name+'_item_embedded_vecs.npy', 'rb') as f:
        item_embedded_vecs = np.load(f, allow_pickle=True)
    with open('./exported_data/'+name+'_HRK_MF.npy', 'rb') as f:
        HRK_MF_SDG = np.load(f, allow_pickle=True)
    with open('./exported_data/'+name+'_HRK_RAND_RECO.npy', 'rb') as f:
        HRK_RAND_RECO = np.load(f, allow_pickle=True)
    with open('./exported_data/'+name+'_HRK_POP_RECO.npy', 'rb') as f:
        HRK_POP_RECO = np.load(f, allow_pickle=True)

        
    return out_recommendation_sets, R_df_as_numpy, [], user_embedded_vecs, item_embedded_vecs, HRK_MF_SDG, HRK_RAND_RECO, HRK_POP_RECO


def export_model_data(name_in, export_hist_removal):
    name = str(name_in)
    with open('./exported_data/'+name+'_QRS_model.item_params.npy', 'wb') as f:
        np.save(f, QRS_model.item_params)
    with open('./exported_data/'+name+'_QRS_model.user_params.npy', 'wb') as f:
        np.save(f, QRS_model.user_params)
    if export_hist_removal:
        with open('./exported_data/'+name+'_QRS_model.hist_removal_params.npy', 'wb') as f:
            np.save(f, QRS_model.hist_removal_params)

            
def load_model_data(name_in, export_hist_removal):
    name = str(name_in)
    with open('./exported_data/'+name+'_QRS_model.item_params.npy', 'rb') as f:
        QRS_model.item_params = np.load(f, allow_pickle=True)
    with open('./exported_data/'+name+'_QRS_model.user_params.npy', 'rb') as f:
        QRS_model.user_params = np.load(f, allow_pickle=True)
    if export_hist_removal:
        with open('./exported_data/'+name+'_QRS_model.hist_removal_params.npy', 'rb') as f:
            QRS_model.hist_removal_params = np.load(f, allow_pickle=True)


In [2]:
LOAD_DATA = 0
LOAD_MODEL_DATA = 1 * LOAD_DATA
LOAD_MODEL_HIST_REMOVAL = 0
export_name = 'temp'
# export_data(export_name)

In [3]:
# ----------------------------- DATA PREPARATION ------------------------------------
HRK_list = []
HRK_list_name = []

random_data = 0

if LOAD_DATA:
    (recommendation_sets, R_df_as_numpy, QRS_init_params, user_embedded_vecs, item_embedded_vecs, HRK_MF_SDG, HRK_RAND_RECO, HRK_POP_RECO) = load_data(export_name)
else:
    # to override the bad interaction in the random data
    # or to override the recommendation sets - use the if below for random data
    if random_data == 1:
        dh = Data_Handler(random_data=random_data)
        dh.add_bad_sample_for_every_user()
        LOO = dh.remove_last_interaction_for_every_user()
        R_df = dh.get_interaction_table()
        recommendation_sets = create_recommendation_sets(LOO, dh)
        R_df_as_numpy = R_df.to_numpy()
    else:
        dh = Data_Handler(random_data=0)
        LOO = dh.remove_last_interaction_for_every_user()
        R_df = dh.get_interaction_table()
        recommendation_sets = create_recommendation_sets(LOO, dh)
        R_df_as_numpy = R_df.to_numpy()




done reading file
done removing latest interactions
num of users: 610
num of movies: 9724
done creating R_df
done creating recommendation sets


In [4]:
visualiser.print_reco_matrix(R_df_as_numpy, user_items_removed_indices)


       0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 9910010110210310410510610710810911011111211311411511611711811912012112212312412512612712812913013113213313413513613713813914014114214314414514614714814915015115215315415515615715815916016116216316416516616716816917017117217317417517617717817918018118218318418518618718818919019119219319419519619719819920020120220320420520620720820921021121221321421521621721821922022122222322422522622722822923023123223323423523623723823924024124224324424524624724824925025125225325425525625725825926026126226326426526626726826927027127227327427527627727827928028128228328428528628728828929029129229329429529629729829930030130230330430530630730830931031131231331431531631731831932032132232332432532632732832933033

2943294429452946294729482949295029512952295329542955295629572958295929602961296229632964296529662967296829692970297129722973297429752976297729782979298029812982298329842985298629872988298929902991299229932994299529962997299829993000300130023003300430053006300730083009301030113012301330143015301630173018301930203021302230233024302530263027302830293030303130323033303430353036303730383039304030413042304330443045304630473048304930503051305230533054305530563057305830593060306130623063306430653066306730683069307030713072307330743075307630773078307930803081308230833084308530863087308830893090309130923093309430953096309730983099310031013102310331043105310631073108310931103111311231133114311531163117311831193120312131223123312431253126312731283129313031313132313331343135313631373138313931403141314231433144314531463147314831493150315131523153315431553156315731583159316031613162316331643165316631673168316931703171317231733174317531763177317831793180318131823183318431853186318731883189319031913192

5943594459455946594759485949595059515952595359545955595659575958595959605961596259635964596559665967596859695970597159725973597459755976597759785979598059815982598359845985598659875988598959905991599259935994599559965997599859996000600160026003600460056006600760086009601060116012601360146015601660176018601960206021602260236024602560266027602860296030603160326033603460356036603760386039604060416042604360446045604660476048604960506051605260536054605560566057605860596060606160626063606460656066606760686069607060716072607360746075607660776078607960806081608260836084608560866087608860896090609160926093609460956096609760986099610061016102610361046105610661076108610961106111611261136114611561166117611861196120612161226123612461256126612761286129613061316132613361346135613661376138613961406141614261436144614561466147614861496150615161526153615461556156615761586159616061616162616361646165616661676168616961706171617261736174617561766177617861796180618161826183618461856186618761886189619061916192

8942894389448945894689478948894989508951895289538954895589568957895889598960896189628963896489658966896789688969897089718972897389748975897689778978897989808981898289838984898589868987898889898990899189928993899489958996899789988999900090019002900390049005900690079008900990109011901290139014901590169017901890199020902190229023902490259026902790289029903090319032903390349035903690379038903990409041904290439044904590469047904890499050905190529053905490559056905790589059906090619062906390649065906690679068906990709071907290739074907590769077907890799080908190829083908490859086908790889089909090919092909390949095909690979098909991009101910291039104910591069107910891099110911191129113911491159116911791189119912091219122912391249125912691279128912991309131913291339134913591369137913891399140914191429143914491459146914791489149915091519152915391549155915691579158915991609161916291639164916591669167916891699170917191729173917491759176917791789179918091819182918391849185918691879188918991909191

  0 |  1  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  

...

  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

  9 |  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  0  0  0 

  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

KeyboardInterrupt: 

In [ ]:
# ------------------------------------------ CLASSIC ALGOS -----------------------------------------
if LOAD_DATA == 0:

    # --------- RANDOM ---------
    RAND_RECO = random_RS()
    HRK_RAND_RECO = TEST_MODEL(recommendation_sets, RAND_RECO)

    # --------- POPULARITY ---------
    POP_RECO = popularity_RS(R_df_as_numpy)
    HRK_POP_RECO = TEST_MODEL(recommendation_sets, POP_RECO)

    
    # ----------- MF -----------
    SDG_MF = MF(R_df_as_numpy, defines._EMBEDDING_SIZE, alpha=0.2, beta=0.01, iterations=500)
    SDG_MF.train()
    HRK_MF_SDG = TEST_MODEL(recommendation_sets, SDG_MF)
#     if random_data == 1:
#         visualiser.print_reco_matrix(R_df_as_numpy, user_items_removed_indices, SDG_MF.full_matrix(), all_positive=0, digits_after_point=2)

    # ------ old MF -----
    #  BinMF = BinaryMF(R_df_as_numpy, defines._EMBEDDING_SIZE, 7000)
    #  BinMF.train()
    #  BinMF.update_predictions()
    #  HRK_MF_BIN = TEST_MODEL(recommendation_sets, BinMF)


HRK_list.append(HRK_RAND_RECO)
HRK_list_name.append("RAND")

HRK_list.append(HRK_POP_RECO)
HRK_list_name.append("POPULARITY")


HRK_list.append(HRK_MF_SDG)
HRK_list_name.append("MF")

visualiser.plot_HRK(HRK_list, HRK_list_name)


In [ ]:
if LOAD_DATA == 0:
    user_embedded_vecs = SDG_MF.get_user_embedded_vectors()
    item_embedded_vecs = SDG_MF.get_item_embedded_vectors()
#     user_embedded_vecs = BinMF.get_user_embedded_vectors()
#     item_embedded_vecs = BinMF.get_item_embedded_vectors()

visualiser.plot_embedded_vecs(user_embedded_vecs)


In [ ]:
if (0):
    QRS_model = embedded_QRS_model4(R_df_as_numpy, train_steps=3, train_sets=10, layers=2, n_groups=4, user_embedding_vec=user_embedded_vecs)
    # QRS_model.train_without_groups()

    # print("\n\nTESTING QRS MODEL")
    # HRK_list.append(TEST_MODEL(recommendation_sets, QRS_model, search_reco_item_within_all_items=1))
    # HRK_list_name.append("QRS")
    # visualiser.plot_HRK(HRK_list, HRK_list_name)

    # QRS_model.destroy_user_params()
    QRS_model.train_with_groups()

    print("\n\nTESTING QRS MODEL")
    HRK_list.append(TEST_MODEL(recommendation_sets, QRS_model, search_reco_item_within_all_items=1))
    HRK_list_name.append("QRS groups")
    visualiser.plot_HRK(HRK_list, HRK_list_name)

    QRS_model.train_hist_removal()

    print("\n\nTESTING QRS MODEL")
    HRK_list.append(TEST_MODEL(recommendation_sets, QRS_model, search_reco_item_within_all_items=1))
    HRK_list_name.append("QRS_hist_removal")
    visualiser.plot_HRK(HRK_list, HRK_list_name)


In [ ]:
model = 'QRS_embedded_layer_item_layer'
if (model==5):
    QRS_model = QRS_user_items(R_df_as_numpy)
if (model=='QRS_user_group_global'):
    QRS_model = QRS_user_group_global_items(R_df_as_numpy, 1)
if (model=='QRS_embedded_layer_item_layer'):
    QRS_model = QRS_embedded_layer_item_layer(R_df_as_numpy, user_embedded_vecs, 5)

    
QRS_model.train()

print("\n\nTESTING QRS MODEL ", model)
HRK_list.append(TEST_MODEL(recommendation_sets, QRS_model, search_reco_item_within_all_items=1))
HRK_list_name.append("QRS")
visualiser.plot_HRK(HRK_list, HRK_list_name)

QRS_model.train_hist_removal()
print("\n\nTESTING QRS MODEL ", model)
HRK_list.append(TEST_MODEL(recommendation_sets, QRS_model, search_reco_item_within_all_items=1))
HRK_list_name.append("QRS_hist_rem")
visualiser.plot_HRK(HRK_list, HRK_list_name)


In [ ]:
export_model_data(export_name, LOAD_MODEL_HIST_REMOVAL)

In [ ]:
STOP!

In [ ]:
models_to_test = []
QRS_train_steps = 10

num_of_layers_list = [10]
if (LOAD_DATA == 0):
    QRS_init_params_list = randomize_init_params(num_of_layers_list)

for index, QRS_init_params in enumerate(QRS_init_params_list):
    for model in models_to_test:
        # model 1 - using embedded vectors as first layer
        if model == 1:
            QRS_model = embedded_QRS_model1(R_df_as_numpy, user_embedded_vecs, item_embedded_vecs, QRS_init_params, train_steps=QRS_train_steps)
        if model == 2:
            QRS_model = embedded_QRS_model2(R_df_as_numpy, train_steps=10, train_sets=10, layers=6)
        if model == 3:
            QRS_model = embedded_QRS_model3(R_df_as_numpy, train_steps=10, train_sets=5, layers=6, n_groups=2)
        if model == 5:
            QRS_model = QRS_user_items(R_df_as_numpy)

        QRS_model.train()

        print("\n\nTESTING QRS MODEL ", model)
        HRK_list.append(TEST_MODEL(recommendation_sets, QRS_model, search_reco_item_within_all_items=1))
        HRK_list_name.append("QRS{}_{}L".format(model, num_of_layers_list[index]))
        visualiser.plot_HRK(HRK_list, HRK_list_name)

        # ------------------------------- HIST REMOVAL ------------------------------
        QRS_reco_matrix = QRS_model.get_QRS_reco_matrix()
        QRS_opt_params = QRS_model.get_QRS_opt_params()
        QRS_HistRem = QRS_hist_removal_per_user(R_df_as_numpy, QRS_reco_matrix, user_embedded_vecs, QRS_opt_params)
        QRS_HistRem.train()

        print("\n\nTESTING QRS HIST REMOVAL ", model)
        HRK_list.append(TEST_MODEL(recommendation_sets, QRS_HistRem, search_reco_item_within_all_items=1))
        HRK_list_name.append("QRS_HistRem{}_{}L".format(model, num_of_layers_list[index]))
        visualiser.plot_HRK(HRK_list, HRK_list_name)

        # ------------------------------- AMPLITUDE AMPLIFY ------------------------------
    #     QRS_HistRem_opt_params = QRS_HistRem.get_QRS_opt_params()
    #     QRS_HistRem_reco_matrix = QRS_HistRem.get_QRS_reco_matrix()

    #     QRS_HistRem_AmpAmp = QRS_amplitude_amp(R_df_as_numpy, QRS_HistRem_reco_matrix, user_embedded_vecs, QRS_opt_params, QRS_HistRem_opt_params)
    #     QRS_HistRem_AmpAmp.train()

    #     print("\n\nTESTING QRS AMP AMP")
    #     HRK_list.append(TEST_MODEL(recommendation_sets, QRS_HistRem_AmpAmp, search_reco_item_within_all_items=1))
    #     HRK_list_name.append("QRS_Amp_{}".format(model))
    #     visualiser.plot_HRK(HRK_list, HRK_list_name)


    


In [ ]:

# ---------------------------------- USING OPTIMIZED ITEMS INDEXING --------------------------
if (0):
    with open('./exported_data/'+str(defines._NUM_OF_QUBITS_TO_OPTIMIZE)+'_opt_score_graph.npy', 'rb') as f:
        opt_score_graph = np.load(f, allow_pickle=True)

    
    user_items_removed_indices = [(x[0], x[1]) for x in recommendation_sets]
    visualiser.print_reco_matrix(R_df_as_numpy, user_items_removed_indices)
    (R_df_as_numpy,recommendation_sets)=improve_entanglemet(R_df_as_numpy,1000, recommendation_sets, 1, opt_score_graph)
    user_items_removed_indices = [(x[0], x[1]) for x in recommendation_sets]
    visualiser.print_reco_matrix(R_df_as_numpy, user_items_removed_indices)


    SDG_MF = MF(R_df_as_numpy, defines._EMBEDDING_SIZE, alpha=0.2, beta=0.01, iterations=50)
    SDG_MF.train()

    HRK_MF_SDG = TEST_MODEL(recommendation_sets, SDG_MF)
    HRK_list.append(HRK_MF_SDG)
    HRK_list_name.append("MF_Ent_Opt")


    user_embedded_vecs = SDG_MF.get_user_embedded_vectors()
    item_embedded_vecs = SDG_MF.get_item_embedded_vectors()



    for index, QRS_init_params in enumerate(QRS_init_params_list):
        for model in models_to_test:
            if model == 1:
                QRS_model = embedded_QRS_model1(R_df_as_numpy, user_embedded_vecs, item_embedded_vecs, QRS_init_params, train_steps=QRS_train_steps)
            if model == 2:
                QRS_model = embedded_QRS_model2(R_df_as_numpy, train_steps=10, train_sets=5, layers=6)
            if model == 3:
                QRS_model = embedded_QRS_model3(R_df_as_numpy, user_embedded_vecs, item_embedded_vecs, QRS_init_params, train_steps=QRS_train_steps)

            QRS_model.train()

            print("\n\nTESTING QRS MODEL ", model)
            HRK_list.append(TEST_MODEL(recommendation_sets, QRS_model, search_reco_item_within_all_items=1))
            HRK_list_name.append("QRS_Ent_Opt{}_{}L".format(model, num_of_layers_list[index]))
            visualiser.plot_HRK(HRK_list, HRK_list_name)

            # ------------------------------- HIST REMOVAL ------------------------------
            QRS_reco_matrix = QRS_model.get_QRS_reco_matrix()
            QRS_opt_params = QRS_model.get_QRS_opt_params()
            QRS_HistRem = QRS_hist_removal_per_user(R_df_as_numpy, QRS_reco_matrix, user_embedded_vecs, QRS_opt_params)
            QRS_HistRem.train()

            print("\n\nTESTING QRS HIST REMOVAL ", model)
            HRK_list.append(TEST_MODEL(recommendation_sets, QRS_HistRem, search_reco_item_within_all_items=1))
            HRK_list_name.append("QRS_Ent_Opt_HistRem{}_{}L".format(model, num_of_layers_list[index]))
            visualiser.plot_HRK(HRK_list, HRK_list_name)



In [ ]:
if (0):
    find_optimized_items_indexing()

In [ ]:
if (1):
    notebook_name = 'run.ipynb'
    output_file_name = '32x128_example.html'

    exporter = HTMLExporter()
    output_notebook = nbformat.read(notebook_name, as_version=4)

    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file_name, 'w', encoding='utf-8').write(output)


In [ ]:
print("Training user:", 1, end="\r")
print("Training user:", 2, end="")